In [1]:
import pandas as pd
import numpy as np
from sklearn import preprocessing

In [5]:
course_genres_df = pd.read_csv('./data/course_genre.csv')
course_genres_df.head()

,COURSE_ID,TITLE,Database,Python,CloudComputing,DataAnalysis,Containers,MachineLearning,ComputerVision,DataScience,BigData,Chatbot,R,BackendDev,FrontendDev,Blockchain
0,ML0201EN,robots are coming build iot apps with watson ...,0,0,0,0,0,0,0,0,0,0,0,1,1,0
1,ML0122EN,accelerating deep learning with gpu,0,1,0,0,0,1,0,1,0,0,0,0,0,0
2,GPXX0ZG0EN,consuming restful services using the reactive ...,0,0,0,0,0,0,0,0,0,0,0,1,1,0
3,RP0105EN,analyzing big data in r using apache spark,1,0,0,1,0,0,0,0,1,0,1,0,0,0
4,GPXX0Z2PEN,containerizing packaging and running a sprin...,0,0,0,0,1,0,0,0,0,0,0,1,0,0


In [67]:
profile_df = pd.read_csv('./data/user_profile.csv')
profile_df.head()

,user,Database,Python,CloudComputing,DataAnalysis,Containers,MachineLearning,ComputerVision,DataScience,BigData,Chatbot,R,BackendDev,FrontendDev,Blockchain
0,2,52.0,14.0,6.0,43.0,3.0,33.0,0.0,29.0,41.0,2.0,18.0,34.0,9.0,6.0
1,4,40.0,2.0,4.0,28.0,0.0,14.0,0.0,20.0,24.0,0.0,6.0,6.0,0.0,2.0
2,5,24.0,8.0,18.0,24.0,0.0,30.0,0.0,22.0,14.0,2.0,14.0,26.0,4.0,6.0
3,7,2.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0
4,8,6.0,0.0,0.0,4.0,0.0,0.0,0.0,0.0,6.0,0.0,2.0,0.0,0.0,0.0


In [26]:
ratings = pd.read_csv('./data/ratings.csv')

In [30]:
courses = ratings[ratings['user']==2103087]['item']

In [61]:
C = course_genres_df[course_genres_df['COURSE_ID'].isin(courses)].iloc[:,2:].to_numpy()

In [62]:
u0 = np.array([[3.0] * len(courses)])
u0

array([[3., 3., 3., 3., 3., 3., 3., 3., 3.]])

In [63]:
print(u0.shape,C.shape)

(1, 9) (9, 14)


In [64]:
u0weights = np.matmul(u0,C)
u0weights

array([[3., 3., 3., 3., 6., 3., 0., 6., 3., 0., 6., 9., 6., 0.]])

In [75]:
u0profile = pd.DataFrame(u0weights.reshape(1,14),columns=course_genres_df.columns[2:]) 
u0profile.insert(0,'user',2103087)
u0profile

,user,Database,Python,CloudComputing,DataAnalysis,Containers,MachineLearning,ComputerVision,DataScience,BigData,Chatbot,R,BackendDev,FrontendDev,Blockchain
0,2103087,3.0,3.0,3.0,3.0,6.0,3.0,0.0,6.0,3.0,0.0,6.0,9.0,6.0,0.0


In [ ]:
enrolled = set(courses)
all_courses = set(course_genres_df['COURSE_ID'].values)
unknown_courses = all_courses.difference(enrolled)
unknown_courses

In [79]:
unknown_courses_genres = course_genres_df[course_genres_df['COURSE_ID'].isin(unknown_courses)]
course_matrix = unknown_courses_genres.iloc[:,2:].values
course_matrix

array([[1, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 1, 0, 0],
       [1, 0, 1, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 1, 0],
       [0, 0, 0, ..., 1, 1, 0],
       [0, 0, 0, ..., 1, 1, 0]], dtype=int64)

In [130]:
test_user_vector = u0weights

In [99]:
len(course_matrix)

298

In [124]:
scores = []
for i in range(298):
    scores.append(np.dot(course_matrix[i].reshape(1,14),u0weights.T)[0,0])

In [136]:
def generate_recommendation_scores(user): 
    enrolled =  ratings[ratings['user']==user]['item'].to_list()
    unknown_courses = all_courses.difference(set(enrolled))
    unknown_courses_df = course_genres_df[course_genres_df['COURSE_ID'].isin(unknown_courses)]
    unknown_courses_ids = unknown_courses_df['COURSE_ID'].values
    
    rows = set(course_genres_df['COURSE_ID']).intersection(set(unknown_courses_ids))
    idx = [(x in rows) for x in course_genres_df['COURSE_ID'].values]
    recommendation_scores = np.dot(course_genres_df[idx].iloc[:,2:].values, test_user_vector.T)
    dict = {}
    for i in range(0,len(unknown_courses_ids)):
        dict[unknown_courses_ids[i]] = recommendation_scores[i][0]
    return dict

In [138]:
user_profile_recommendation = generate_recommendation_scores(2103087)

In [149]:
sorted_dict = dict(sorted(user_profile_recommendation.items(),key=lambda x: x[1],reverse=True))

In [157]:
recommended_courses = list(sorted_dict.keys())[:10]

In [159]:
course_genres_df[course_genres_df['COURSE_ID'].isin(recommended_courses)]   

,COURSE_ID,TITLE,Database,Python,CloudComputing,DataAnalysis,Containers,MachineLearning,ComputerVision,DataScience,BigData,Chatbot,R,BackendDev,FrontendDev,Blockchain
10,GPXX0SDXEN,testing microservices with the arquillian mana...,0,0,0,0,1,0,0,0,0,0,0,1,0,0
22,OS0101EN,introduction to open source,0,0,0,0,0,0,0,0,0,0,0,1,1,0
44,SW0201EN,how to build watson ai and swift apis and make...,0,0,1,0,0,1,0,0,0,0,0,1,0,0
49,SW0101EN,build swift mobile apps with watson ai services,0,0,1,0,0,1,0,0,0,0,0,1,0,0
56,CO0401EN,beyond the basics istio and ibm cloud kuberne...,0,0,1,0,1,0,0,0,0,0,0,1,0,0
98,GPXX04V3EN,deploy a web server using python and ibm cloud...,0,1,1,0,0,0,0,0,0,0,0,1,0,0
100,GPXX0D14EN,build a personal movie recommender with django,0,1,0,0,0,1,0,0,0,0,0,1,0,0
207,CC0150EN,building cloud native and multicloud applications,0,0,1,0,0,0,0,0,0,0,0,1,1,0
256,excourse43,cloud virtualization containers and apis,0,0,1,0,1,0,0,0,0,0,0,1,0,0
257,excourse44,alibaba cloud native solutions and container s...,0,0,1,0,1,0,0,0,0,0,0,1,0,0
